In [1]:
import numpy as np
import pandas as pd

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *



#################################             FUNCTIONNALITY             ########################################


# this programs reads a csv file from the TBF110. A year and a month are required to launch this program

# From the year and month selected, the program counts the sum ITEMS_TOTAL_AMOUNT for each sender_id and each days

# It creates 0 values for the non existing dates for each sender_id and stores it in a csv for later visualizing


#################################################################################################################



# Choose your year and your month

yearChosen = 2017
monthChosen = 7


# Set up the spark session

spark = SparkSession.builder \
   .master("spark://spark-master:7077") \
   .appName("sumbyday") \
   .config(conf = pyspark.SparkConf()) \
   .getOrCreate()



# Read the csv File

df1 = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('../sparkJup/romain.csv')


dftemp = df1.filter((df1['year'] == yearChosen) & (df1['month'] == monthChosen)).groupby("day","SENDER_SITE_LOC_ID").agg(sum('ITEMS_TOTAL_AMOUNT'))
dftemp.show()


# Transforms to Pandas Dataframe

dfDaysForYearAndMonthChosen = dftemp.toPandas()



i = 0

# Remove the duplicates for sender_id

senderId = pd.unique(dfDaysForYearAndMonthChosen['SENDER_SITE_LOC_ID'].values)

# A table with all the dates which occur one time

dateByDays = [t for t in range(1,32)]

# Set up the table length

lenDateByDays = len(dateByDays)
lenSenderId = len(senderId)

print(senderId)
print(dateByDays)

#print(lenDateByYear)
#print(lenSenderId)


# This loop is used to create some rows for sender_id which did not have all the dates
# Example sender_id = 1 days = 2,15,20 dateByYear=[1,2,3,..,29,20,31]
# A new row is created to have sender_id = 1 days 1, sender_id = 1 days = 3 etc
# This is compulsory for creating visualization

while i < lenSenderId:
    for j in range(lenDateByDays):
        if i+j < lenSenderId + lenDateByDays:
            dfBoolean = dfDaysForYearAndMonthChosen.loc[dfDaysForYearAndMonthChosen['SENDER_SITE_LOC_ID'].isin([senderId[i]]) & dfDaysForYearAndMonthChosen['day'].isin([dateByDays[j]])]
            #print(senderId[i])
            #print(dateByDays[j])
            #print(dftemp)
            if (dfBoolean.empty == True):
                df3 = pd.DataFrame(np.array([[dateByDays[j],senderId[i]]]), columns=['day','SENDER_SITE_LOC_ID'])
                dfDaysForYearAndMonthChosen = dfDaysForYearAndMonthChosen.append(df3, ignore_index=True)
#                #print(dfsum['SENDER_SITE_LOC_ID'][i])
#                #print(dfsum['year'][i])
#                print(tab2[j])
#                print('not ok')
            else :
                print('ok')
                #print(dfsum['SENDER_SITE_LOC_ID'][i])
                #print(dfsum['year'][i])
                #print(tab2[j])


        else:
            if(dfsum['month'][i+j - 1] != dateByDays[len(dateByDays)-1]):
                #print('make something')
                df3 = pd.DataFrame(np.array([[dateByDays[j],senderId[i]]]), columns=['day','SENDER_SITE_LOC_ID'])
                dfDaysForYearAndMonthChosen = dfDaysForYearAndMonthChosen.append(df3, ignore_index=True)
    i = i + 1


print(dfDaysForYearAndMonthChosen)


# Replace the values which are NaN by 0
dfDaysForYearAndMonthChosen= dfDaysForYearAndMonthChosen.fillna(0)

#print(dfsum)

# Save to a csv file
dfDaysForYearAndMonthChosen.to_csv('sumbyday.csv')

spark.stop()



+---+------------------+-----------------------+
|day|SENDER_SITE_LOC_ID|sum(ITEMS_TOTAL_AMOUNT)|
+---+------------------+-----------------------+
| 18|             15695|               12730720|
| 28|             15695|                 222560|
| 11|             15793|               11336936|
| 19|             15695|                  25992|
| 18|             15801|            12531848256|
| 18|              8627|            28521692304|
| 18|             15793|            51075796380|
| 17|              7597|              524884448|
+---+------------------+-----------------------+

[15695 15793 15801  8627  7597]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
ok
ok
ok
ok
ok
ok
ok
ok
     SENDER_SITE_LOC_ID  day  sum(ITEMS_TOTAL_AMOUNT)
0                 15695   18             1.273072e+07
1                 15695   28             2.225600e+05
2                 15793   11             1.133694e+07
3                 15695